In [1]:
# ----------------------------------------------------------------------
# Adaptive AI Trading Robot Architecture (Conceptual)
# ----------------------------------------------------------------------
# This script sets up the structure for a continuous-learning trading agent
# using the MT5 Python API. The core logic in the RLStrategy class is
# a placeholder where a real Reinforcement Learning or Adaptive Model
# (e.g., Q-Learning, Deep Q-Network) would be implemented.
# ----------------------------------------------------------------------

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime
import time
import random

# --- CONFIGURATION ---
SYMBOL = "Volatility 100 Index"
TRADE_VOLUME = 1.0  
TRAINING_BARS = 200 # Bars to fetch for initial training
LOOKBACK_PERIOD = 20 # How many bars the AI looks at for a prediction
TICK_INTERVAL_SEC = 5 # Frequency of the main loop check (seconds)
TIMEFRAME = mt5.TIMEFRAME_M5 # Use 5-minute bars for training/data

# --- GLOBAL MT5 OBJECTS ---
MT5_LOGIN = None 

# ----------------------------------------------------------------------
# 1. AI Strategy Class (The Learning Agent)
# ----------------------------------------------------------------------

class RLStrategy:
    """
    Conceptual Reinforcement Learning Agent.
    In a real application, this would use libraries like stable-baselines3 or TensorFlow.
    """
    def __init__(self):
        # State: Represents the model's internal knowledge (weights/parameters)
        # For simplicity, we use a simple average-based rule as a placeholder.
        self.model_state = {'average_gain': 0.0}
        print("AI Strategy initialized (Placeholder mode).")
        
    def train(self, historical_data, last_trade_pnl=None):
        """
        Simulates the model training/adaptation phase.
        
        :param historical_data: The current dataset (OHLC).
        :param last_trade_pnl: The reward signal from the last trade.
        """
        
        # --- (PLACEHOLDER RL TRAINING LOGIC) ---
        
        if last_trade_pnl is not None:
            # Simple simulation: If the last trade was profitable, slightly adjust the "average_gain" target up.
            if last_trade_pnl > 0:
                self.model_state['average_gain'] = 0.5 * self.model_state['average_gain'] + 0.5 * 0.01 
            else:
                self.model_state['average_gain'] = 0.5 * self.model_state['average_gain'] - 0.5 * 0.01

            print(f"   [AI Trained] Last PnL: {last_trade_pnl:.2f}. New internal state updated.")

        # In a real RL system, 'historical_data' would be used to replay experiences
        # and update the underlying neural network weights here.
        
    def predict(self, current_data):
        """
        Analyzes the market data and predicts the next action.
        
        :param current_data: The last N bars of market data (a DataFrame).
        :return: mt5.ORDER_TYPE_BUY, mt5.ORDER_TYPE_SELL, or 'HOLD'.
        """
        if current_data.empty or len(current_data) < 2:
            return 'HOLD'

        # --- (PLACEHOLDER PREDICTION LOGIC) ---
        # Strategy: Buy if the last close is higher than the mean of the lookback period,
        # weighted by the current 'average_gain' from training.

        close_prices = current_data['close'].values
        mean_price = np.mean(close_prices)
        last_close = close_prices[-1]
        
        # Decision boundary is adjusted by the model state
        decision_threshold = mean_price * (1 + self.model_state['average_gain'])

        if last_close > decision_threshold:
            print("   [AI Prediction] BUY Signal (Price > Adjusted Mean)")
            return mt5.ORDER_TYPE_BUY
        elif last_close < mean_price * (1 - self.model_state['average_gain']):
            print("   [AI Prediction] SELL Signal (Price < Adjusted Mean)")
            return mt5.ORDER_TYPE_SELL
        else:
            print("   [AI Prediction] HOLD Signal (No clear conviction)")
            return 'HOLD'

# ----------------------------------------------------------------------
# 2. Trade Manager Class (Handles MT5 Interaction)
# ----------------------------------------------------------------------

class TradeManager:
    
    def __init__(self, symbol, volume):
        self.symbol = symbol
        self.volume = volume
        self.last_trade_deal = None # Stores the ticket of the last executed deal
        self.open_position_ticket = None # Stores the ticket of the current open position
        self.open_position_type = None # Stores the type of the current open position
        self.trades_executed = 0

    def fetch_ohlc_data(self, timeframe, n_bars):
        """Fetches the last N bars for the symbol."""
        rates = mt5.copy_rates_from_pos(self.symbol, timeframe, 0, n_bars)
        if rates is None or len(rates) == 0:
            print(f"!!! Error fetching data: {mt5.last_error()}")
            return pd.DataFrame()
        
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        return df[['time', 'open', 'high', 'low', 'close', 'tick_volume']].set_index('time')

    def place_trade(self, trade_type):
        """Submits an order to the connected MT5 terminal."""
        
        # Check if there is an existing position to avoid multiple positions
        if self.open_position_ticket is not None:
            print(f"   [Trade Manager] Cannot open {trade_type} trade. Position {self.open_position_ticket} is already open.")
            return False

        # Get the latest tick/price information
        tick = mt5.symbol_info_tick(self.symbol)
        if tick is None or tick.bid <= 0 or tick.ask <= 0:
            print("!!! Error: Invalid tick data retrieved. Cannot trade.")
            return False
            
        if trade_type == mt5.ORDER_TYPE_BUY:
            price = tick.ask
            order_type = mt5.ORDER_TYPE_BUY
            action = mt5.TRADE_ACTION_DEAL
        else: # SELL
            price = tick.bid
            order_type = mt5.ORDER_TYPE_SELL
            action = mt5.TRADE_ACTION_DEAL

        request = {
            "action": action,
            "symbol": self.symbol,
            "volume": self.volume,
            "type": order_type,
            "price": price,
            "deviation": 20, 
            "magic": 202409, 
            "comment": "AI_Adaptive_Bot",
            "type_time": mt5.ORDER_TIME_GTC, 
            "type_filling": mt5.ORDER_FILLING_FOK,
        }
        
        result = mt5.order_send(request)
        
        if result is None:
            print("!!! CRITICAL: mt5.order_send() returned None. Check terminal connection.")
            return False
        
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"!!! Order Send Failed. Code: {result.retcode}. Comment: {result.comment}")
            return False
        else:
            print(f"*** TRADE EXECUTED *** | Ticket: {result.deal} | Type: {trade_type} | Price: {result.price}")
            self.open_position_ticket = result.deal
            self.open_position_type = trade_type
            self.trades_executed += 1
            return True

    def check_and_close_position(self):
        """
        Checks the status of the current open position and closes it based on a simple PnL rule.
        Also calculates the PnL to be used as a reward signal for the AI.
        
        :return: PnL of the closed trade, or None if no trade was closed.
        """
        if self.open_position_ticket is None:
            return None

        positions = mt5.positions_get(symbol=self.symbol)
        
        # Find the open position for PnL calculation
        current_position = next((p for p in positions if p.ticket == self.open_position_ticket), None)

        if current_position is None:
            # The position might have been closed manually or by SL/TP
            print(f"   [Trade Manager] Position {self.open_position_ticket} not found (Closed externally).")
            pnl_reward = random.uniform(-10.0, 10.0) # Assume some PnL if closed externally
            self.open_position_ticket = None
            self.open_position_type = None
            return pnl_reward


        # --- SIMPLE EXIT LOGIC: Close if PnL is > $1 or < -$5 (hardcoded for demo) ---
        pnl_threshold_profit = 1.0 
        pnl_threshold_loss = -5.0
        
        if current_position.profit > pnl_threshold_profit or current_position.profit < pnl_threshold_loss:
            
            # Prepare close request
            close_type = mt5.ORDER_TYPE_SELL if current_position.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
            tick = mt5.symbol_info_tick(self.symbol)
            close_price = tick.bid if current_position.type == mt5.ORDER_TYPE_BUY else tick.ask

            close_request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": self.symbol,
                "volume": current_position.volume,
                "type": close_type,
                "position": current_position.ticket,
                "price": close_price,
                "deviation": 20,
                "magic": 202409,
                "comment": "AI_Exit_Signal",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_FOK,
            }

            close_result = mt5.order_send(close_request)
            
            if close_result is None or close_result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"!!! Error closing position {current_position.ticket}. RetCode: {close_result.retcode if close_result else 'None'}")
                return None
            else:
                pnl_reward = current_position.profit
                print(f"*** POSITION CLOSED *** | Ticket: {current_position.ticket} | PnL: {pnl_reward:.2f} (Reward Signal)")
                self.open_position_ticket = None
                self.open_position_type = None
                return pnl_reward

        # Position is still open and within bounds
        print(f"   [Position Update] Ticket {current_position.ticket} | PnL: {current_position.profit:.2f}")
        return None

# ----------------------------------------------------------------------
# 3. Main Script Execution
# ----------------------------------------------------------------------

if __name__ == "__main__":
    
    # --- Initialization ---
    if not mt5.initialize():
        print(f"!!! MT5 Initialization FAILED. Check if terminal is running: {mt5.last_error()}")
        exit()

    if mt5.account_info() is None:
        print("!!! FAILED TO CONNECT to a logged-in account.")
        mt5.shutdown()
        exit()
        
    print(f"MT5 initialized and connected automatically. Account: {mt5.account_info().login}")
    
    # Create instances of the manager and the AI
    manager = TradeManager(SYMBOL, TRADE_VOLUME)
    ai_strategy = RLStrategy()
    
    last_pnl_reward = None
    loop_count = 0

    print(f"\nStarting Adaptive Trading Loop for {SYMBOL}...")
    
    while loop_count < 20: # Run for 20 simulation cycles
        
        loop_count += 1
        print(f"\n--- CYCLE {loop_count:02d} | Time: {datetime.now().strftime('%H:%M:%S')} ---")
        
        # 1. Fetch Data
        # We fetch enough data for the training window
        data = manager.fetch_ohlc_data(TIMEFRAME, TRAINING_BARS)
        
        if data.empty:
            print("!!! Data fetch failed. Skipping cycle.")
            time.sleep(TICK_INTERVAL_SEC)
            continue
            
        # 2. Train/Adapt the AI (using the PnL reward from the last cycle)
        print("   [Training] Running adaptive training based on last PnL...")
        ai_strategy.train(data, last_pnl_reward)
        
        # 3. Get Prediction (using the latest lookback period data)
        latest_data_subset = data.tail(LOOKBACK_PERIOD)
        signal = ai_strategy.predict(latest_data_subset)
        
        # 4. Check for exit/close signal
        pnl_closed = manager.check_and_close_position()
        
        if pnl_closed is not None:
            # If a trade was closed, update the reward signal for the next training cycle
            last_pnl_reward = pnl_closed
            
        # 5. Execute Trade
        if signal != 'HOLD' and manager.open_position_ticket is None:
            manager.place_trade(signal)
            
        # 6. Wait for the next interval
        print(f"   [Status] Waiting {TICK_INTERVAL_SEC} seconds...")
        time.sleep(TICK_INTERVAL_SEC)
        
    # --- Cleanup ---
    print("\n--- SIMULATION ENDED ---")
    
    # Final check: Close any remaining open position
    if manager.open_position_ticket is not None:
        print(f"Attempting to close final position {manager.open_position_ticket}...")
        manager.check_and_close_position() 

    mt5.shutdown()
    print(f"Total trades executed: {manager.trades_executed}")

MT5 initialized and connected automatically. Account: 40866995
AI Strategy initialized (Placeholder mode).

Starting Adaptive Trading Loop for Volatility 100 Index...

--- CYCLE 01 | Time: 22:47:20 ---
   [Training] Running adaptive training based on last PnL...
   [AI Prediction] SELL Signal (Price < Adjusted Mean)
*** TRADE EXECUTED *** | Ticket: 4431304321 | Type: 1 | Price: 864.62
   [Status] Waiting 5 seconds...

--- CYCLE 02 | Time: 22:47:28 ---
   [Training] Running adaptive training based on last PnL...
   [AI Prediction] SELL Signal (Price < Adjusted Mean)
   [Trade Manager] Position 4431304321 not found (Closed externally).
*** TRADE EXECUTED *** | Ticket: 4431304422 | Type: 1 | Price: 864.8
   [Status] Waiting 5 seconds...

--- CYCLE 03 | Time: 22:47:34 ---
   [Training] Running adaptive training based on last PnL...
   [AI Trained] Last PnL: 2.81. New internal state updated.
   [AI Prediction] HOLD Signal (No clear conviction)
   [Trade Manager] Position 4431304422 not foun

In [3]:
# ----------------------------------------------------------------------
# Adaptive AI Trading Robot Architecture (Conceptual)
# ----------------------------------------------------------------------
# This script sets up the structure for a continuous-learning trading agent
# using the MT5 Python API. The core logic in the RLStrategy class is
# a placeholder where a real Reinforcement Learning or Adaptive Model
# (e.g., Q-Learning, Deep Q-Network) would be implemented.
# ----------------------------------------------------------------------

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime
import time
import random

# --- CONFIGURATION ---
SYMBOL = "Volatility 100 Index"
TRADE_VOLUME = 1.0  
TRAINING_BARS = 200 # Bars to fetch for initial training
LOOKBACK_PERIOD = 20 # How many bars the AI looks at for a prediction
TICK_INTERVAL_SEC = 5 # Frequency of the main loop check (seconds)
TIMEFRAME = mt5.TIMEFRAME_M5 # Use 5-minute bars for training/data

# --- GLOBAL MT5 OBJECTS ---
MT5_LOGIN = None 

# ----------------------------------------------------------------------
# 1. AI Strategy Class (The Learning Agent)
# ----------------------------------------------------------------------

class RLStrategy:
    """
    Conceptual Reinforcement Learning Agent.
    In a real application, this would use libraries like stable-baselines3 or TensorFlow.
    """
    def __init__(self):
        # State: Represents the model's internal knowledge (weights/parameters)
        # For simplicity, we use a simple average-based rule as a placeholder.
        self.model_state = {'average_gain': 0.0, 'macd_preference': 0.5} # Added macd_preference
        print("AI Strategy initialized (Placeholder mode).")
        
    def train(self, historical_data, last_trade_pnl=None):
        """
        Simulates the model training/adaptation phase.
        
        :param historical_data: The current dataset (OHLC + Indicators).
        :param last_trade_pnl: The reward signal from the last trade.
        """
        
        # --- (PLACEHOLDER RL TRAINING LOGIC) ---
        
        if last_trade_pnl is not None:
            # Simple simulation: Adjust internal state based on profit/loss
            reward_factor = 0.01 * np.sign(last_trade_pnl)
            
            # Adjust 'average_gain' (related to mean reversion)
            self.model_state['average_gain'] = max(0, self.model_state['average_gain'] + reward_factor)
            
            # Adjust 'macd_preference' (related to momentum)
            # Example adaptation: if trade was profitable, increase preference for momentum signals
            self.model_state['macd_preference'] = np.clip(self.model_state['macd_preference'] + reward_factor, 0.1, 0.9)


            print(f"   [AI Trained] Last PnL: {last_trade_pnl:.2f}. New internal state updated.")

        # In a real RL system, 'historical_data' would be used to replay experiences
        # and update the underlying neural network weights here.
        
    def predict(self, current_data):
        """
        Analyzes the market data and predicts the next action.
        
        :param current_data: The last N bars of market data (a DataFrame with indicators).
        :return: mt5.ORDER_TYPE_BUY, mt5.ORDER_TYPE_SELL, or 'HOLD'.
        """
        if current_data.empty or len(current_data) < 2 or 'MACD' not in current_data.columns:
            return 'HOLD'

        # --- (ADAPTED PREDICTION LOGIC - Uses Indicators) ---
        
        # 1. Mean Reversion Signal (from original logic, adapted by average_gain)
        close_prices = current_data['close'].values
        mean_price = np.mean(close_prices)
        last_close = close_prices[-1]
        
        reversion_buy_signal = last_close < mean_price * (1 - self.model_state['average_gain'])
        reversion_sell_signal = last_close > mean_price * (1 + self.model_state['average_gain'])

        # 2. Momentum Signal (based on MACD Crossover, using latest data point)
        last_macd = current_data['MACD'].iloc[-1]
        last_signal = current_data['Signal_Line'].iloc[-1]
        
        momentum_buy_signal = last_macd > last_signal # MACD crosses above Signal Line
        momentum_sell_signal = last_macd < last_signal # MACD crosses below Signal Line
        
        # 3. Combine Signals based on learned 'macd_preference' weight
        # Total Conviction = (Momentum Conviction * macd_preference) + (Reversion Conviction * (1 - macd_preference))
        
        buy_conviction = (momentum_buy_signal * self.model_state['macd_preference']) + \
                         (reversion_buy_signal * (1 - self.model_state['macd_preference']))
                         
        sell_conviction = (momentum_sell_signal * self.model_state['macd_preference']) + \
                          (reversion_sell_signal * (1 - self.model_state['macd_preference']))
                          
        # Note: In this simple boolean arithmetic, True=1 and False=0.
        
        if buy_conviction > sell_conviction and buy_conviction > 0:
            print(f"   [AI Prediction] BUY Signal (Conviction: {buy_conviction:.2f})")
            return mt5.ORDER_TYPE_BUY
        elif sell_conviction > buy_conviction and sell_conviction > 0:
            print(f"   [AI Prediction] SELL Signal (Conviction: {sell_conviction:.2f})")
            return mt5.ORDER_TYPE_SELL
        else:
            print("   [AI Prediction] HOLD Signal (No clear conviction)")
            return 'HOLD'

# ----------------------------------------------------------------------
# 2. Trade Manager Class (Handles MT5 Interaction & Data Processing)
# ----------------------------------------------------------------------

class TradeManager:
    
    def __init__(self, symbol, volume):
        self.symbol = symbol
        self.volume = volume
        self.last_trade_deal = None # Stores the ticket of the last executed deal
        self.open_position_ticket = None # Stores the ticket of the current open position
        self.open_position_type = None # Stores the type of the current open position
        self.trades_executed = 0

    def fetch_ohlc_data(self, timeframe, n_bars):
        """Fetches the last N bars for the symbol."""
        rates = mt5.copy_rates_from_pos(self.symbol, timeframe, 0, n_bars)
        if rates is None or len(rates) == 0:
            print(f"!!! Error fetching data: {mt5.last_error()}")
            return pd.DataFrame()
        
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        return df[['time', 'open', 'high', 'low', 'close', 'tick_volume']].set_index('time')

    def add_technical_indicators(self, df):
        """
        Calculates and adds key technical indicators (MACD, RSI) to the DataFrame.
        """
        if df.empty:
            return df

        # --- MACD Calculation ---
        # 12-period EMA
        ema_12 = df['close'].ewm(span=12, adjust=False).mean()
        # 26-period EMA
        ema_26 = df['close'].ewm(span=26, adjust=False).mean()
        # MACD Line
        df['MACD'] = ema_12 - ema_26
        # Signal Line (9-period EMA of MACD)
        df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()
        # MACD Histogram is not strictly needed for the signal but is good for visualization
        df['MACD_Hist'] = df['MACD'] - df['Signal_Line']

        # --- RSI Calculation (14-period) ---
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))

        return df.dropna() # Drop initial rows where indicators are not fully calculated


    def place_trade(self, trade_type):
        """Submits an order to the connected MT5 terminal."""
        
        # Check if there is an existing position to avoid multiple positions
        if self.open_position_ticket is not None:
            print(f"   [Trade Manager] Cannot open {trade_type} trade. Position {self.open_position_ticket} is already open.")
            return False

        # Get the latest tick/price information
        tick = mt5.symbol_info_tick(self.symbol)
        if tick is None or tick.bid <= 0 or tick.ask <= 0:
            print("!!! Error: Invalid tick data retrieved. Cannot trade.")
            return False
            
        if trade_type == mt5.ORDER_TYPE_BUY:
            price = tick.ask
            order_type = mt5.ORDER_TYPE_BUY
            action = mt5.TRADE_ACTION_DEAL
        else: # SELL
            price = tick.bid
            order_type = mt5.ORDER_TYPE_SELL
            action = mt5.TRADE_ACTION_DEAL

        request = {
            "action": action,
            "symbol": self.symbol,
            "volume": self.volume,
            "type": order_type,
            "price": price,
            "deviation": 20, 
            "magic": 202409, 
            "comment": "AI_Adaptive_Bot",
            "type_time": mt5.ORDER_TIME_GTC, 
            "type_filling": mt5.ORDER_FILLING_FOK,
        }
        
        result = mt5.order_send(request)
        
        if result is None:
            print("!!! CRITICAL: mt5.order_send() returned None. Check terminal connection.")
            return False
        
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"!!! Order Send Failed. Code: {result.retcode}. Comment: {result.comment}")
            return False
        else:
            print(f"*** TRADE EXECUTED *** | Ticket: {result.deal} | Type: {'BUY' if trade_type == mt5.ORDER_TYPE_BUY else 'SELL'} | Price: {result.price}")
            self.open_position_ticket = result.deal
            self.open_position_type = trade_type
            self.trades_executed += 1
            return True

    def check_and_close_position(self):
        """
        Checks the status of the current open position and closes it based on a simple PnL rule.
        Also calculates the PnL to be used as a reward signal for the AI.
        
        :return: PnL of the closed trade, or None if no trade was closed.
        """
        if self.open_position_ticket is None:
            return None

        positions = mt5.positions_get(symbol=self.symbol)
        
        # Find the open position for PnL calculation
        current_position = next((p for p in positions if p.ticket == self.open_position_ticket), None)

        if current_position is None:
            # The position might have been closed manually or by SL/TP
            print(f"   [Trade Manager] Position {self.open_position_ticket} not found (Closed externally).")
            # In a real system, you would try to fetch the deal history to get the actual PnL.
            # For this simulation, we'll assume a random outcome.
            pnl_reward = random.uniform(-10.0, 10.0)
            self.open_position_ticket = None
            self.open_position_type = None
            return pnl_reward


        # --- SIMPLE EXIT LOGIC: Close if PnL is > $1 or < -$5 (hardcoded for demo) ---
        pnl_threshold_profit = 1.0 
        pnl_threshold_loss = -5.0
        
        if current_position.profit > pnl_threshold_profit or current_position.profit < pnl_threshold_loss:
            
            # Prepare close request
            close_type = mt5.ORDER_TYPE_SELL if current_position.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
            tick = mt5.symbol_info_tick(self.symbol)
            close_price = tick.bid if current_position.type == mt5.ORDER_TYPE_BUY else tick.ask

            close_request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": self.symbol,
                "volume": current_position.volume,
                "type": close_type,
                "position": current_position.ticket,
                "price": close_price,
                "deviation": 20,
                "magic": 202409,
                "comment": "AI_Exit_Signal",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_FOK,
            }

            close_result = mt5.order_send(close_request)
            
            if close_result is None or close_result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"!!! Error closing position {current_position.ticket}. RetCode: {close_result.retcode if close_result else 'None'}")
                return None
            else:
                pnl_reward = current_position.profit
                print(f"*** POSITION CLOSED *** | Ticket: {current_position.ticket} | PnL: {pnl_reward:.2f} (Reward Signal)")
                self.open_position_ticket = None
                self.open_position_type = None
                return pnl_reward

        # Position is still open and within bounds
        print(f"   [Position Update] Ticket {current_position.ticket} | PnL: {current_position.profit:.2f}")
        return None

# ----------------------------------------------------------------------
# 3. Main Script Execution
# ----------------------------------------------------------------------

if __name__ == "__main__":
    
    # --- Initialization ---
    if not mt5.initialize():
        print(f"!!! MT5 Initialization FAILED. Check if terminal is running: {mt5.last_error()}")
        exit()

    if mt5.account_info() is None:
        print("!!! FAILED TO CONNECT to a logged-in account.")
        mt5.shutdown()
        exit()
        
    print(f"MT5 initialized and connected automatically. Account: {mt5.account_info().login}")
    
    # Create instances of the manager and the AI
    manager = TradeManager(SYMBOL, TRADE_VOLUME)
    ai_strategy = RLStrategy()
    
    last_pnl_reward = None
    loop_count = 0

    print(f"\nStarting Adaptive Trading Loop for {SYMBOL}...")
    
    while loop_count < 20: # Run for 20 simulation cycles
        
        loop_count += 1
        print(f"\n--- CYCLE {loop_count:02d} | Time: {datetime.now().strftime('%H:%M:%S')} ---")
        
        # 1. Fetch Data
        # We fetch enough data for the training window
        raw_data = manager.fetch_ohlc_data(TIMEFRAME, TRAINING_BARS)
        
        if raw_data.empty:
            print("!!! Data fetch failed. Skipping cycle.")
            time.sleep(TICK_INTERVAL_SEC)
            continue
            
        # 2. Feature Engineering: Add Technical Indicators
        data_with_indicators = manager.add_technical_indicators(raw_data.copy())
        
        if data_with_indicators.empty:
             print("!!! Data is too short for indicator calculation. Skipping cycle.")
             time.sleep(TICK_INTERVAL_SEC)
             continue
             
        # 3. Train/Adapt the AI (using the PnL reward from the last cycle)
        print("   [Training] Running adaptive training based on last PnL...")
        # Note: In a real model, we would use the entire 'data_with_indicators' for deep learning
        ai_strategy.train(data_with_indicators, last_pnl_reward)
        
        # 4. Get Prediction (using the latest lookback period data)
        # We use the lookback period *after* indicators have been calculated and NaNs dropped
        latest_data_subset = data_with_indicators.tail(LOOKBACK_PERIOD)
        signal = ai_strategy.predict(latest_data_subset)
        
        # 5. Check for exit/close signal
        pnl_closed = manager.check_and_close_position()
        
        if pnl_closed is not None:
            # If a trade was closed, update the reward signal for the next training cycle
            last_pnl_reward = pnl_closed
            
        # 6. Execute Trade
        if signal != 'HOLD' and manager.open_position_ticket is None:
            manager.place_trade(signal)
            
        # 7. Wait for the next interval
        print(f"   [Status] Waiting {TICK_INTERVAL_SEC} seconds...")
        time.sleep(TICK_INTERVAL_SEC)
        
    # --- Cleanup ---
    print("\n--- SIMULATION ENDED ---")
    
    # Final check: Close any remaining open position
    if manager.open_position_ticket is not None:
        print(f"Attempting to close final position {manager.open_position_ticket}...")
        manager.check_and_close_position() 

    mt5.shutdown()
    print(f"Total trades executed: {manager.trades_executed}")

MT5 initialized and connected automatically. Account: 40866995
AI Strategy initialized (Placeholder mode).

Starting Adaptive Trading Loop for Volatility 100 Index...

--- CYCLE 01 | Time: 23:14:24 ---
   [Training] Running adaptive training based on last PnL...
   [AI Prediction] HOLD Signal (No clear conviction)
   [Status] Waiting 5 seconds...

--- CYCLE 02 | Time: 23:14:29 ---
   [Training] Running adaptive training based on last PnL...
   [AI Prediction] HOLD Signal (No clear conviction)
   [Status] Waiting 5 seconds...

--- CYCLE 03 | Time: 23:14:34 ---
   [Training] Running adaptive training based on last PnL...
   [AI Prediction] HOLD Signal (No clear conviction)
   [Status] Waiting 5 seconds...

--- CYCLE 04 | Time: 23:14:39 ---
   [Training] Running adaptive training based on last PnL...
   [AI Prediction] HOLD Signal (No clear conviction)
   [Status] Waiting 5 seconds...

--- CYCLE 05 | Time: 23:14:44 ---
   [Training] Running adaptive training based on last PnL...
   [AI Pr